<a href="https://colab.research.google.com/github/dineshhn/MinutesOfMeeting/blob/updated_mom/MinutesOfMeeting_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/dineshhn/MinutesOfMeeting/blob/updated_mom/MinutesOfMeeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install all the dependencies. Place any new installables into requirements.txt file.
!pip install -r /content/drive/MyDrive/requirements.txt

In [ ]:
import torch

In [ ]:
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Select an audio file and read it:
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
audio_sample = ds[0]["audio"]

# Load the Whisper model with SDPA attention
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en", attn_implementation="sdpa")

# Enable static cache and compile the forward pass
model.generation_config.cache_implementation = "static"
model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)

# Use the model and processor to transcribe the audio:
input_features = processor(
    audio_sample["array"], sampling_rate=audio_sample["sampling_rate"], return_tensors="pt"
).input_features

# Compile the forward pass
for _ in range(2):
    model.generate(input_features)

# Generate token ids using compiled graph (fast!)
predicted_ids = model.generate(input_features)

# Decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

transcription[0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


' Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.'

In [ ]:
from transformers import AutoProcessor, Wav2Vec2BertForCTC
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation", trust_remote_code=True)
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

processor = AutoProcessor.from_pretrained("hf-audio/wav2vec2-bert-CV16-en")
model = Wav2Vec2BertForCTC.from_pretrained("hf-audio/wav2vec2-bert-CV16-en")

# audio file is decoded on the fly
inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)

# transcribe speech
transcription = processor.batch_decode(predicted_ids)
transcription[0]

inputs["labels"] = processor(text=dataset[0]["text"], return_tensors="pt").input_ids

# compute loss
loss = model(**inputs).loss
round(loss.item(), 2)

17.04

In [ ]:
import torch
from transformers import pipeline

whisper = pipeline("automatic-speech-recognition", "openai/whisper-large-v3", torch_dtype=torch.float16, device="cuda:0")

# transcription = whisper("/content/drive/MyDrive/ColabNotebooks/EarningsCall.wav", return_timestamps=True)

# print(transcription["text"])

Device set to use cuda:0


In [ ]:
!pip install deepspeech

ERROR: Could not find a version that satisfies the requirement deepspeech (from versions: none)
ERROR: No matching distribution found for deepspeech


In [ ]:
# --- Transcription & Summary Function ---
def transcribe_audio_with_whisper(file):
    whisper = pipeline(
        "automatic-speech-recognition",
        "openai/whisper-large-v3",
        torch_dtype=torch.float16,
        device="cuda:0"
    )

    start_trans = time.time()
    transcription = whisper(file, return_timestamps=True)
    end_trans = time.time()
    full_transcription = transcription["text"]
    trans_time = f"Transcription time: {end_trans - start_trans:.2f} seconds"

    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    start_summ = time.time()
    summary = summarizer(full_transcription, max_length=130, min_length=30, do_sample=False)[0]["summary_text"]
    end_summ = time.time()
    summ_time = f"Summarization time: {end_summ - start_summ:.2f} seconds"

    return full_transcription, summary, trans_time, summ_time

In [ ]:
# Dispatcher
def process_based_on_selection(file, choice):
    if choice == "Whisper-large-v3":
        return transcribe_audio_with_whisper(file)
    else:
        return "Invalid model", "Invalid model", "", ""

In [ ]:
def extract_action_items(full_transcription, keywords):
    # Use default keywords if none provided
    if not keywords:
        keywords = [
            "must", "should", "need to", "investing", "expanding",
            "diversifying", "enhancing", "revenue", "risk", "loss"
        ]
    else:
        keywords = [kw.strip().lower() for kw in keywords.split(",")]

    # Split transcription into sentences
    sentences = re.split(r'(?<=[.!?])\s+', full_transcription)

    # Filter sentences containing any keyword
    action_items = [
        sentence for sentence in sentences
        if any(keyword in sentence.lower() for keyword in keywords)
    ]

    return action_items


In [ ]:
def process_action_items(file, choice, keywords):
    transcription, _, _, _ = process_based_on_selection(file, choice)

    start_action = time.time()
    action_items = extract_action_items(transcription, keywords)
    end_action = time.time()
    action_time = f"Action item extraction time: {end_action - start_action:.2f} seconds"

    if action_items:
        formatted_actions = "\n".join([f"{i+1}. {item}" for i, item in enumerate(action_items)])
    else:
        formatted_actions = "No action items found."

    return f"{formatted_actions}\n\n{action_time}"


In [ ]:
def process_and_format_outputs(file, choice, keywords):
    transcription, summary, trans_time, summ_time = process_based_on_selection(file, choice)

    # Format transcription and summary
    trans_with_time = f"{transcription}\n\n{trans_time}"
    summ_with_time = f"{summary}\n\n{summ_time}"
    both_times = f"{trans_time}\n{summ_time}"

    # Action items
    start_action = time.time()
    action_items = extract_action_items(transcription, keywords)
    end_action = time.time()
    action_time = f"Action item extraction time: {end_action - start_action:.2f} seconds"

    if action_items:
        formatted_actions = "\n".join([f"{i+1}. {item}" for i, item in enumerate(action_items)])
    else:
        formatted_actions = "No action items found."
    actions_with_time = f"{formatted_actions}\n\n{action_time}"

    return trans_with_time, summ_with_time, both_times, actions_with_time


In [ ]:
# def generate_txt_report(transcription, summary, trans_time, summ_time, action_items_text, metrics_text):
#     report = (
#         " Minutes of Meeting Report\n"
#         "============================\n\n"
#         f"Transcription:\n{transcription}\n\n"
#         f"{trans_time}\n\n"
#         f"Summary:\n{summary}\n\n"
#         f"{summ_time}\n\n"
#         f" Action Items:\n{action_items_text}\n\n"
#         f" Evaluation Metrics:\n{metrics_text}\n"
#     )

#     # Save to file
#     file_path = "/mnt/data/minutes_report.txt"
#     with open(file_path, "w", encoding="utf-8") as f:
#         f.write(report)

#     return file_path  # Return the path so it can be used as a downloadable file


In [ ]:
!pip install gradio
import gradio as gr # Import the gradio library and alias it as 'gr'
from transformers import pipeline
import torch
import time
import re
with gr.Blocks() as demo:
    gr.Markdown("### Minutes of meeting")

    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(label="Upload your audio file")
            dropdown = gr.Dropdown(choices=["Whisper-large-v3", "Facebook - Wav2vec2", "Whisper-Small", "Mozzilla - Deepspeech", "Google - Speech Recognition"], label="Select Model")
            keyword_input = gr.Textbox(
                label="Custom Action Item Keywords (comma-separated)",
                placeholder="e.g. must, should, expanding, revenue"
            )
            with gr.Row():
                btn_trans = gr.Button("Transcribe the uploaded file")
                btn_summ = gr.Button("Generate the summary")
            #btn_time = gr.Button("Show Both Timings")
            btn_action = gr.Button("Extract Action Items")
            btn_all = gr.Button("Process All")

        with gr.Column(scale=1):
            output_trans = gr.Textbox(label="Transcribed Text", lines=10)
            output_summ = gr.Textbox(label="Summary", lines=6)
            output_time = gr.Textbox(label="Processing time", lines=2)
            output_action = gr.Textbox(label="Action Items", lines=10)
            output_metrics = gr.Textbox(label="Evaluation Metrics", lines=10)


    # Individual buttons
    btn_trans.click(
        fn=lambda file, choice: f"{process_based_on_selection(file, choice)[0]}\n\n{process_based_on_selection(file, choice)[2]}",
        inputs=[file_input, dropdown],
        outputs=output_trans
    )

    btn_summ.click(
        fn=lambda file, choice: f"{process_based_on_selection(file, choice)[1]}\n\n{process_based_on_selection(file, choice)[3]}",
        inputs=[file_input, dropdown],
        outputs=output_summ
    )

    # btn_time.click(
    #     fn=lambda file, choice: f"{process_based_on_selection(file, choice)[2]}\n{process_based_on_selection(file, choice)[3]}",
    #     inputs=[file_input, dropdown],
    #     outputs=output_time
    # )

    btn_action.click(
        fn=process_action_items,
        inputs=[file_input, dropdown, keyword_input],
        outputs=output_action
    )

    # Combined "Process All" button
    btn_all.click(
        fn=process_and_format_outputs,
        inputs=[file_input, dropdown, keyword_input],
        outputs=[output_trans, output_summ, output_time, output_action, output_metrics]
    )

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://da9a9a92961aa08c12.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from rouge_score import rouge_scorer

nltk.download('punkt')

def evaluate_summary(transcription, summary):
    # Tokenize
    ref_tokens = nltk.word_tokenize(transcription.lower())
    gen_tokens = nltk.word_tokenize(summary.lower())

    # BLEU Score
    bleu = sentence_bleu([ref_tokens], gen_tokens)

    # Align lengths for token-wise metrics
    min_len = min(len(ref_tokens), len(gen_tokens))
    y_true = ref_tokens[:min_len]
    y_pred = gen_tokens[:min_len]

    precision = precision_score(y_true, y_pred, average='micro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='micro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='micro', zero_division=0)
    accuracy = accuracy_score(y_true, y_pred)

    # ROUGE Score
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(transcription, summary)
    rouge1 = rouge_scores['rouge1'].fmeasure
    rougel = rouge_scores['rougeL'].fmeasure

    metrics = (
        f"Evaluation Metrics:\n\n"
        f"BLEU Score: {bleu:.4f}\n"
        f"Precision: {precision:.4f}\n"
        f"Recall: {recall:.4f}\n"
        f"F1 Score: {f1:.4f}\n"
        f"Accuracy: {accuracy:.4f}\n"
        f"ROUGE-1: {rouge1:.4f}\n"
        f"ROUGE-L: {rougel:.4f}"
    )

    return metrics


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def process_and_format_outputs(file, choice, keywords):
    transcription, summary, trans_time, summ_time = process_based_on_selection(file, choice)

    trans_with_time = f"{transcription}\n\n{trans_time}"
    summ_with_time = f"{summary}\n\n{summ_time}"
    both_times = f"{trans_time}\n{summ_time}"

    # Action items
    start_action = time.time()
    action_items = extract_action_items(transcription, keywords)
    end_action = time.time()
    action_time = f"Action item extraction time: {end_action - start_action:.2f} seconds"

    if action_items:
        formatted_actions = "\n".join([f"{i+1}. {item}" for i, item in enumerate(action_items)])
    else:
        formatted_actions = "No action items found."
    actions_with_time = f"{formatted_actions}\n\n{action_time}"

    # Evaluate summary
    evaluation_results = evaluate_summary(transcription, summary)

    return trans_with_time, summ_with_time, both_times, actions_with_time, evaluation_results
